# Unsupervised Language Learning, Lab1 - Word Analogy

## Adriaan de Vries (10795227), Verna Dankers (10761225)

Before being able to run this code, please import the following libraries and set the following paths to the datasets. Afterwards, the code should run without issues.

In [55]:
# Requirements
from tqdm import tqdm
from pprint import pprint
from collections import defaultdict, Counter
from scipy.stats import spearmanr, pearsonr
from tabulate import tabulate
from gensim.models import KeyedVectors
from gensim.corpora.dictionary import Dictionary
from gensim.scripts.glove2word2vec import glove2word2vec
import numpy as np
import os
import matplotlib.pyplot as plt

# Paths to datasets
bow2_filename = "data/bow2.words"
bow5_filename = "data/bow5.words"
deps_filename = "data/deps.words"
simlex_filename = "data/SimLex-999.txt"
men_filename = "data/men/MEN_dataset_natural_form_full"
analogy_filename = "data/questions-words.txt"
common_words_filename = "data/common_words.words"
create_figures = False

In [57]:
# Read in word embeddings
glove2word2vec(bow2_filename, bow2_filename.split(".")[0] + ".txt")
glove2word2vec(bow5_filename, bow5_filename.split(".")[0] + ".txt")
glove2word2vec(deps_filename, deps_filename.split(".")[0] + ".txt")
bow2 = KeyedVectors.load_word2vec_format(bow2_filename.split(".")[0] + ".txt", binary=False)
bow2.init_sims(replace=True)
bow5 = KeyedVectors.load_word2vec_format(bow5_filename.split(".")[0] + ".txt", binary=False)
bow5.init_sims(replace=True)
deps = KeyedVectors.load_word2vec_format(deps_filename.split(".")[0] + ".txt", binary=False)
deps.init_sims(replace=True)

## Quantitative Analysis

Excluding nothing:

<table>
<thead>
<tr><th style="text-align: right;">  Amount</th><th>Topic                      </th><th style="text-align: right;">  Acc. Bow2</th><th style="text-align: right;">  MRR Bow2</th><th style="text-align: right;">  Acc. Bow5</th><th style="text-align: right;">  MRR Bow5</th><th style="text-align: right;">  Acc. Deps</th><th style="text-align: right;">  MRR Deps</th></tr>
</thead>
<tbody>
<tr><td style="text-align: right;">     866</td><td>currency                   </td><td style="text-align: right;">0.0123839  </td><td style="text-align: right;"> 0.0856626</td><td style="text-align: right;"> 0.0309598 </td><td style="text-align: right;">  0.103925</td><td style="text-align: right;"> 0.0100671 </td><td style="text-align: right;"> 0.0611891</td></tr>
<tr><td style="text-align: right;">    1332</td><td>gram8-plural               </td><td style="text-align: right;">0.0127628  </td><td style="text-align: right;"> 0.403051 </td><td style="text-align: right;"> 0.0247748 </td><td style="text-align: right;">  0.398332</td><td style="text-align: right;"> 0.0315315 </td><td style="text-align: right;"> 0.399549 </td></tr>
<tr><td style="text-align: right;">     506</td><td>family                     </td><td style="text-align: right;">0.12253    </td><td style="text-align: right;"> 0.491784 </td><td style="text-align: right;"> 0.171937  </td><td style="text-align: right;">  0.522562</td><td style="text-align: right;"> 0.0909091 </td><td style="text-align: right;"> 0.478123 </td></tr>
<tr><td style="text-align: right;">    1560</td><td>gram7-past-tense           </td><td style="text-align: right;">0.0192308  </td><td style="text-align: right;"> 0.349355 </td><td style="text-align: right;"> 0.0211538 </td><td style="text-align: right;">  0.361567</td><td style="text-align: right;"> 0.0487179 </td><td style="text-align: right;"> 0.401535 </td></tr>
<tr><td style="text-align: right;">    1599</td><td>gram6-nationality-adjective</td><td style="text-align: right;">0.526579   </td><td style="text-align: right;"> 0.678273 </td><td style="text-align: right;"> 0.595372  </td><td style="text-align: right;">  0.741515</td><td style="text-align: right;"> 0.00562852</td><td style="text-align: right;"> 0.137225 </td></tr>
<tr><td style="text-align: right;">    2467</td><td>city-in-state              </td><td style="text-align: right;">0.000810701</td><td style="text-align: right;"> 0.197824 </td><td style="text-align: right;"> 0.0113498 </td><td style="text-align: right;">  0.309294</td><td style="text-align: right;"> 0         </td><td style="text-align: right;"> 0.10959  </td></tr>
<tr><td style="text-align: right;">    1122</td><td>gram4-superlative          </td><td style="text-align: right;">0.0388258  </td><td style="text-align: right;"> 0.375839 </td><td style="text-align: right;"> 0.0445076 </td><td style="text-align: right;">  0.357888</td><td style="text-align: right;"> 0.0501894 </td><td style="text-align: right;"> 0.330852 </td></tr>
<tr><td style="text-align: right;">     812</td><td>gram2-opposite             </td><td style="text-align: right;">0.00985222 </td><td style="text-align: right;"> 0.212035 </td><td style="text-align: right;"> 0.0172414 </td><td style="text-align: right;">  0.218784</td><td style="text-align: right;"> 0         </td><td style="text-align: right;"> 0.246448 </td></tr>
<tr><td style="text-align: right;">    1056</td><td>gram5-present-participle   </td><td style="text-align: right;">0.0388258  </td><td style="text-align: right;"> 0.399059 </td><td style="text-align: right;"> 0.0265152 </td><td style="text-align: right;">  0.417497</td><td style="text-align: right;"> 0.0350379 </td><td style="text-align: right;"> 0.402661 </td></tr>
<tr><td style="text-align: right;">     992</td><td>gram1-adjective-to-adverb  </td><td style="text-align: right;">0.00201613 </td><td style="text-align: right;"> 0.113949 </td><td style="text-align: right;"> 0.00504032</td><td style="text-align: right;">  0.140235</td><td style="text-align: right;"> 0.00504032</td><td style="text-align: right;"> 0.0425688</td></tr>
<tr><td style="text-align: right;">     870</td><td>gram9-plural-verbs         </td><td style="text-align: right;">0.109195   </td><td style="text-align: right;"> 0.491881 </td><td style="text-align: right;"> 0.116092  </td><td style="text-align: right;">  0.481422</td><td style="text-align: right;"> 0.147126  </td><td style="text-align: right;"> 0.552559 </td></tr>
<tr><td style="text-align: right;">    1332</td><td>gram3-comparative          </td><td style="text-align: right;">0.130631   </td><td style="text-align: right;"> 0.53767  </td><td style="text-align: right;"> 0.198198  </td><td style="text-align: right;">  0.549162</td><td style="text-align: right;"> 0.0788288 </td><td style="text-align: right;"> 0.461443 </td></tr>
<tr><td style="text-align: right;">    4524</td><td>capital-world              </td><td style="text-align: right;">0.0596817  </td><td style="text-align: right;"> 0.375586 </td><td style="text-align: right;"> 0.0587975 </td><td style="text-align: right;">  0.440326</td><td style="text-align: right;"> 0.00176835</td><td style="text-align: right;"> 0.107777 </td></tr>
<tr><td style="text-align: right;">     506</td><td>capital-common-countries   </td><td style="text-align: right;">0.229249   </td><td style="text-align: right;"> 0.546255 </td><td style="text-align: right;"> 0.22332   </td><td style="text-align: right;">  0.595877</td><td style="text-align: right;"> 0.00197628</td><td style="text-align: right;"> 0.214714 </td></tr>
<tr><td style="text-align: right;">   19544</td><td>All                        </td><td style="text-align: right;">0.0886904  </td><td style="text-align: right;"> 0.372925 </td><td style="text-align: right;"> 0.103386  </td><td style="text-align: right;">  0.41304 </td><td style="text-align: right;"> 0.0268638 </td><td style="text-align: right;"> 0.241322 </td></tr>
</tbody>
</table>

Excluding a, b, a*:

<table>
<thead>
<tr><th style="text-align: right;">  Amount</th><th>Topic                      </th><th style="text-align: right;">  Acc. Bow2</th><th style="text-align: right;">  MRR Bow2</th><th style="text-align: right;">  Acc. Bow5</th><th style="text-align: right;">  MRR Bow5</th><th style="text-align: right;">  Acc. Deps</th><th style="text-align: right;">  MRR Deps</th></tr>
</thead>
<tbody>
<tr><td style="text-align: right;">     866</td><td>currency                   </td><td style="text-align: right;">       0.11</td><td style="text-align: right;">      0.15</td><td style="text-align: right;">       0.12</td><td style="text-align: right;">      0.17</td><td style="text-align: right;">       0.06</td><td style="text-align: right;">      0.1 </td></tr>
<tr><td style="text-align: right;">    1332</td><td>gram8-plural               </td><td style="text-align: right;">       0.73</td><td style="text-align: right;">      0.79</td><td style="text-align: right;">       0.67</td><td style="text-align: right;">      0.75</td><td style="text-align: right;">       0.68</td><td style="text-align: right;">      0.75</td></tr>
<tr><td style="text-align: right;">     506</td><td>family                     </td><td style="text-align: right;">       0.79</td><td style="text-align: right;">      0.85</td><td style="text-align: right;">       0.82</td><td style="text-align: right;">      0.87</td><td style="text-align: right;">       0.82</td><td style="text-align: right;">      0.85</td></tr>
<tr><td style="text-align: right;">    1560</td><td>gram7-past-tense           </td><td style="text-align: right;">       0.56</td><td style="text-align: right;">      0.66</td><td style="text-align: right;">       0.55</td><td style="text-align: right;">      0.67</td><td style="text-align: right;">       0.66</td><td style="text-align: right;">      0.73</td></tr>
<tr><td style="text-align: right;">    1599</td><td>gram6-nationality-adjective</td><td style="text-align: right;">       0.74</td><td style="text-align: right;">      0.81</td><td style="text-align: right;">       0.82</td><td style="text-align: right;">      0.86</td><td style="text-align: right;">       0.12</td><td style="text-align: right;">      0.22</td></tr>
<tr><td style="text-align: right;">    2467</td><td>city-in-state              </td><td style="text-align: right;">       0.39</td><td style="text-align: right;">      0.5 </td><td style="text-align: right;">       0.51</td><td style="text-align: right;">      0.62</td><td style="text-align: right;">       0.12</td><td style="text-align: right;">      0.22</td></tr>
<tr><td style="text-align: right;">    1122</td><td>gram4-superlative          </td><td style="text-align: right;">       0.63</td><td style="text-align: right;">      0.73</td><td style="text-align: right;">       0.57</td><td style="text-align: right;">      0.7 </td><td style="text-align: right;">       0.56</td><td style="text-align: right;">      0.64</td></tr>
<tr><td style="text-align: right;">     812</td><td>gram2-opposite             </td><td style="text-align: right;">       0.36</td><td style="text-align: right;">      0.42</td><td style="text-align: right;">       0.36</td><td style="text-align: right;">      0.43</td><td style="text-align: right;">       0.4 </td><td style="text-align: right;">      0.48</td></tr>
<tr><td style="text-align: right;">    1056</td><td>gram5-present-participle   </td><td style="text-align: right;">       0.63</td><td style="text-align: right;">      0.75</td><td style="text-align: right;">       0.67</td><td style="text-align: right;">      0.78</td><td style="text-align: right;">       0.65</td><td style="text-align: right;">      0.74</td></tr>
<tr><td style="text-align: right;">     992</td><td>gram1-adjective-to-adverb  </td><td style="text-align: right;">       0.16</td><td style="text-align: right;">      0.24</td><td style="text-align: right;">       0.17</td><td style="text-align: right;">      0.27</td><td style="text-align: right;">       0.03</td><td style="text-align: right;">      0.07</td></tr>
<tr><td style="text-align: right;">     870</td><td>gram9-plural-verbs         </td><td style="text-align: right;">       0.81</td><td style="text-align: right;">      0.86</td><td style="text-align: right;">       0.74</td><td style="text-align: right;">      0.82</td><td style="text-align: right;">       0.91</td><td style="text-align: right;">      0.94</td></tr>
<tr><td style="text-align: right;">    1332</td><td>gram3-comparative          </td><td style="text-align: right;">       0.9 </td><td style="text-align: right;">      0.94</td><td style="text-align: right;">       0.83</td><td style="text-align: right;">      0.89</td><td style="text-align: right;">       0.8 </td><td style="text-align: right;">      0.85</td></tr>
<tr><td style="text-align: right;">    4524</td><td>capital-world              </td><td style="text-align: right;">       0.63</td><td style="text-align: right;">      0.72</td><td style="text-align: right;">       0.7 </td><td style="text-align: right;">      0.8 </td><td style="text-align: right;">       0.11</td><td style="text-align: right;">      0.2 </td></tr>
<tr><td style="text-align: right;">     506</td><td>capital-common-countries   </td><td style="text-align: right;">       0.84</td><td style="text-align: right;">      0.88</td><td style="text-align: right;">       0.94</td><td style="text-align: right;">      0.96</td><td style="text-align: right;">       0.35</td><td style="text-align: right;">      0.49</td></tr>
<tr><td style="text-align: right;">   19544</td><td>All                        </td><td style="text-align: right;">       0.59</td><td style="text-align: right;">      0.67</td><td style="text-align: right;">       0.62</td><td style="text-align: right;">      0.71</td><td style="text-align: right;">       0.37</td><td style="text-align: right;">      0.45</td></tr>
</tbody>
</table>

## Qualitative Analysis

Excluding nothing:

<table>
<thead>
<tr><th>Topic               </th><th>A          </th><th>A*           </th><th>B           </th><th>B*            </th><th>Bow2        </th><th>Bow5        </th><th>Deps        </th></tr>
</thead>
<tbody>
<tr><td>currency            </td><td>armenia    </td><td>dram         </td><td>korea       </td><td>won           </td><td>dram        </td><td>dram        </td><td>dram        </td></tr>
<tr><td>currency            </td><td>korea      </td><td>won          </td><td>iran        </td><td>rial          </td><td>won         </td><td>won         </td><td>won         </td></tr>
<tr><td>currency            </td><td>europe     </td><td>euro         </td><td>iran        </td><td>rial          </td><td>euro        </td><td>euro        </td><td>euro        </td></tr>
<tr><td>gram8-plural        </td><td>road       </td><td>roads        </td><td>bird        </td><td>birds         </td><td>bird        </td><td>bird        </td><td>bird        </td></tr>
<tr><td>gram8-plural        </td><td>melon      </td><td>melons       </td><td>cloud       </td><td>clouds        </td><td>cloud       </td><td>cloud       </td><td>cloud       </td></tr>
<tr><td>gram8-plural        </td><td>eagle      </td><td>eagles       </td><td>car         </td><td>cars          </td><td>car         </td><td>car         </td><td>car         </td></tr>
<tr><td>family              </td><td>grandpa    </td><td>grandma      </td><td>king        </td><td>queen         </td><td>king        </td><td>king        </td><td>king        </td></tr>
<tr><td>family              </td><td>sons       </td><td>daughters    </td><td>stepfather  </td><td>stepmother    </td><td>stepfather  </td><td>stepfather  </td><td>stepfather  </td></tr>
<tr><td>family              </td><td>stepson    </td><td>stepdaughter </td><td>uncle       </td><td>aunt          </td><td>uncle       </td><td>uncle       </td><td>uncle       </td></tr>
<tr><td>gram7-past-tense    </td><td>describing </td><td>described    </td><td>generating  </td><td>generated     </td><td>generating  </td><td>generating  </td><td>generating  </td></tr>
<tr><td>gram7-past-tense    </td><td>taking     </td><td>took         </td><td>hiding      </td><td>hid           </td><td>hiding      </td><td>hiding      </td><td>hiding      </td></tr>
<tr><td>gram7-past-tense    </td><td>dancing    </td><td>danced       </td><td>slowing     </td><td>slowed        </td><td>slowing     </td><td>slowing     </td><td>slowing     </td></tr>
<tr><td>gram6-nationality-ad</td><td>india      </td><td>indian       </td><td>portugal    </td><td>portuguese    </td><td>indian      </td><td>portugal    </td><td>indian      </td></tr>
<tr><td>gram6-nationality-ad</td><td>macedonia  </td><td>macedonian   </td><td>ireland     </td><td>irish         </td><td>ireland     </td><td>ireland     </td><td>macedonian  </td></tr>
<tr><td>gram6-nationality-ad</td><td>australia  </td><td>australian   </td><td>france      </td><td>french        </td><td>australian  </td><td>france      </td><td>australian  </td></tr>
<tr><td>city-in-state       </td><td>atlanta    </td><td>georgia      </td><td>cincinnati  </td><td>ohio          </td><td>georgia     </td><td>georgia     </td><td>georgia     </td></tr>
<tr><td>city-in-state       </td><td>omaha      </td><td>nebraska     </td><td>bakersfield </td><td>california    </td><td>bakersfield </td><td>bakersfield </td><td>nebraska    </td></tr>
<tr><td>city-in-state       </td><td>denver     </td><td>colorado     </td><td>pittsburgh  </td><td>pennsylvania  </td><td>colorado    </td><td>pittsburgh  </td><td>colorado    </td></tr>
<tr><td>gram4-superlative   </td><td>young      </td><td>youngest     </td><td>smart       </td><td>smartest      </td><td>smart       </td><td>smart       </td><td>youngest    </td></tr>
<tr><td>gram4-superlative   </td><td>wide       </td><td>widest       </td><td>cold        </td><td>coldest       </td><td>cold        </td><td>cold        </td><td>cold        </td></tr>
<tr><td>gram4-superlative   </td><td>bright     </td><td>brightest    </td><td>small       </td><td>smallest      </td><td>small       </td><td>small       </td><td>small       </td></tr>
<tr><td>gram2-opposite      </td><td>likely     </td><td>unlikely     </td><td>clear       </td><td>unclear       </td><td>clear       </td><td>clear       </td><td>clear       </td></tr>
<tr><td>gram2-opposite      </td><td>acceptable </td><td>unacceptable </td><td>decided     </td><td>undecided     </td><td>decided     </td><td>decided     </td><td>decided     </td></tr>
<tr><td>gram2-opposite      </td><td>clear      </td><td>unclear      </td><td>possibly    </td><td>impossibly    </td><td>possibly    </td><td>possibly    </td><td>possibly    </td></tr>
<tr><td>gram5-present-partic</td><td>read       </td><td>reading      </td><td>increase    </td><td>increasing    </td><td>increase    </td><td>increase    </td><td>increase    </td></tr>
<tr><td>gram5-present-partic</td><td>debug      </td><td>debugging    </td><td>play        </td><td>playing       </td><td>play        </td><td>play        </td><td>play        </td></tr>
<tr><td>gram5-present-partic</td><td>slow       </td><td>slowing      </td><td>shuffle     </td><td>shuffling     </td><td>shuffle     </td><td>shuffle     </td><td>shuffle     </td></tr>
<tr><td>gram1-adjective-to-a</td><td>complete   </td><td>completely   </td><td>professional</td><td>professionally</td><td>professional</td><td>professional</td><td>professional</td></tr>
<tr><td>gram1-adjective-to-a</td><td>possible   </td><td>possibly     </td><td>immediate   </td><td>immediately   </td><td>immediate   </td><td>immediate   </td><td>possibly    </td></tr>
<tr><td>gram1-adjective-to-a</td><td>unfortunate</td><td>unfortunately</td><td>professional</td><td>professionally</td><td>professional</td><td>professional</td><td>professional</td></tr>
<tr><td>gram9-plural-verbs  </td><td>think      </td><td>thinks       </td><td>play        </td><td>plays         </td><td>play        </td><td>play        </td><td>play        </td></tr>
<tr><td>gram9-plural-verbs  </td><td>increase   </td><td>increases    </td><td>listen      </td><td>listens       </td><td>listen      </td><td>listen      </td><td>listen      </td></tr>
<tr><td>gram9-plural-verbs  </td><td>sit        </td><td>sits         </td><td>swim        </td><td>swims         </td><td>swim        </td><td>swim        </td><td>swim        </td></tr>
<tr><td>gram3-comparative   </td><td>great      </td><td>greater      </td><td>simple      </td><td>simpler       </td><td>simple      </td><td>simple      </td><td>simple      </td></tr>
<tr><td>gram3-comparative   </td><td>slow       </td><td>slower       </td><td>great       </td><td>greater       </td><td>great       </td><td>great       </td><td>great       </td></tr>
<tr><td>gram3-comparative   </td><td>big        </td><td>bigger       </td><td>old         </td><td>older         </td><td>old         </td><td>old         </td><td>old         </td></tr>
<tr><td>capital-world       </td><td>muscat     </td><td>oman         </td><td>stockholm   </td><td>sweden        </td><td>stockholm   </td><td>stockholm   </td><td>stockholm   </td></tr>
<tr><td>capital-world       </td><td>rabat      </td><td>morocco      </td><td>tashkent    </td><td>uzbekistan    </td><td>tashkent    </td><td>tashkent    </td><td>morocco     </td></tr>
<tr><td>capital-world       </td><td>maputo     </td><td>mozambique   </td><td>muscat      </td><td>oman          </td><td>muscat      </td><td>muscat      </td><td>muscat      </td></tr>
<tr><td>capital-common-count</td><td>bangkok    </td><td>thailand     </td><td>beijing     </td><td>china         </td><td>beijing     </td><td>beijing     </td><td>thailand    </td></tr>
<tr><td>capital-common-count</td><td>baghdad    </td><td>iraq         </td><td>ottawa      </td><td>canada        </td><td>ottawa      </td><td>ottawa      </td><td>ottawa      </td></tr>
<tr><td>capital-common-count</td><td>baghdad    </td><td>iraq         </td><td>bern        </td><td>switzerland   </td><td>bern        </td><td>bern        </td><td>bern        </td></tr>
</tbody>
</table>

Excluding a, b and a*:

<table>
<thead>
<tr><th>Topic               </th><th>A          </th><th>A*           </th><th>B           </th><th>B*            </th><th>Bow2       </th><th>Bow5              </th><th>Deps             </th></tr>
</thead>
<tbody>
<tr><td>currency            </td><td>armenia    </td><td>dram         </td><td>korea       </td><td>won           </td><td>sram       </td><td>sram              </td><td>sram             </td></tr>
<tr><td>currency            </td><td>korea      </td><td>won          </td><td>iran        </td><td>rial          </td><td>clinched   </td><td>winning           </td><td>clinched         </td></tr>
<tr><td>currency            </td><td>europe     </td><td>euro         </td><td>iran        </td><td>rial          </td><td>reza       </td><td>saveh             </td><td>baht             </td></tr>
<tr><td>gram8-plural        </td><td>dollar     </td><td>dollars      </td><td>man         </td><td>men           </td><td>woman      </td><td>woman             </td><td>woman            </td></tr>
<tr><td>gram8-plural        </td><td>cat        </td><td>cats         </td><td>banana      </td><td>bananas       </td><td>yams       </td><td>avocados          </td><td>cherries         </td></tr>
<tr><td>gram8-plural        </td><td>computer   </td><td>computers    </td><td>woman       </td><td>women         </td><td>girl       </td><td>man               </td><td>man              </td></tr>
<tr><td>family              </td><td>father     </td><td>mother       </td><td>groom       </td><td>bride         </td><td>bridegroom </td><td>bridegroom        </td><td>bridegroom       </td></tr>
<tr><td>family              </td><td>grandpa    </td><td>grandma      </td><td>groom       </td><td>bride         </td><td>bridegroom </td><td>bridegroom        </td><td>bridegroom       </td></tr>
<tr><td>family              </td><td>boy        </td><td>girl         </td><td>husband     </td><td>wife          </td><td>fianc      </td><td>fianc             </td><td>boyfriend        </td></tr>
<tr><td>gram7-past-tense    </td><td>thinking   </td><td>thought      </td><td>knowing     </td><td>knew          </td><td>believed   </td><td>believed          </td><td>believed         </td></tr>
<tr><td>gram7-past-tense    </td><td>vanishing  </td><td>vanished     </td><td>seeing      </td><td>saw           </td><td>disappeared</td><td>disappeared       </td><td>lurked           </td></tr>
<tr><td>gram7-past-tense    </td><td>decreasing </td><td>decreased    </td><td>seeing      </td><td>saw           </td><td>noticing   </td><td>noticing          </td><td>marveled         </td></tr>
<tr><td>gram6-nationality-ad</td><td>denmark    </td><td>danish       </td><td>england     </td><td>english       </td><td>scottish   </td><td>welsh             </td><td>swedish          </td></tr>
<tr><td>gram6-nationality-ad</td><td>germany    </td><td>german       </td><td>argentina   </td><td>argentinean   </td><td>argentine  </td><td>argentine         </td><td>italian          </td></tr>
<tr><td>gram6-nationality-ad</td><td>switzerland</td><td>swiss        </td><td>belarus     </td><td>belorussian   </td><td>ukrainian  </td><td>belarusian        </td><td>algerian         </td></tr>
<tr><td>city-in-state       </td><td>omaha      </td><td>nebraska     </td><td>bakersfield </td><td>california    </td><td>fresno     </td><td>fresno            </td><td>illinois         </td></tr>
<tr><td>city-in-state       </td><td>fresno     </td><td>california   </td><td>garland     </td><td>texas         </td><td>tuchman    </td><td>judy              </td><td>posy             </td></tr>
<tr><td>city-in-state       </td><td>reno       </td><td>nevada       </td><td>spokane     </td><td>washington    </td><td>oregon     </td><td>yakima            </td><td>oregon           </td></tr>
<tr><td>gram4-superlative   </td><td>young      </td><td>youngest     </td><td>smart       </td><td>smartest      </td><td>oldest     </td><td>eldest            </td><td>safest           </td></tr>
<tr><td>gram4-superlative   </td><td>easy       </td><td>easiest      </td><td>good        </td><td>best          </td><td>nicest     </td><td>bad               </td><td>nicest           </td></tr>
<tr><td>gram4-superlative   </td><td>large      </td><td>largest      </td><td>weak        </td><td>weakest       </td><td>strongest  </td><td>strongest         </td><td>strongest        </td></tr>
<tr><td>gram2-opposite      </td><td>clear      </td><td>unclear      </td><td>possibly    </td><td>impossibly    </td><td>perhaps    </td><td>uncertain         </td><td>probably         </td></tr>
<tr><td>gram2-opposite      </td><td>honest     </td><td>dishonest    </td><td>fortunate   </td><td>unfortunate   </td><td>fortuitous </td><td>troublesome       </td><td>imprudent        </td></tr>
<tr><td>gram2-opposite      </td><td>fortunate  </td><td>unfortunate  </td><td>possibly    </td><td>impossibly    </td><td>perhaps    </td><td>oversimplification</td><td>perhaps          </td></tr>
<tr><td>gram5-present-partic</td><td>read       </td><td>reading      </td><td>increase    </td><td>increasing    </td><td>decrease   </td><td>decrease          </td><td>decrease         </td></tr>
<tr><td>gram5-present-partic</td><td>say        </td><td>saying       </td><td>think       </td><td>thinking      </td><td>stating    </td><td>stating           </td><td>intimating       </td></tr>
<tr><td>gram5-present-partic</td><td>read       </td><td>reading      </td><td>vanish      </td><td>vanishing     </td><td>disappear  </td><td>disappear         </td><td>disappear        </td></tr>
<tr><td>gram1-adjective-to-a</td><td>complete   </td><td>completely   </td><td>professional</td><td>professionally</td><td>totally    </td><td>totally           </td><td>totally          </td></tr>
<tr><td>gram1-adjective-to-a</td><td>possible   </td><td>possibly     </td><td>immediate   </td><td>immediately   </td><td>probably   </td><td>perhaps           </td><td>probably         </td></tr>
<tr><td>gram1-adjective-to-a</td><td>unfortunate</td><td>unfortunately</td><td>slow        </td><td>slowly        </td><td>slower     </td><td>slowed            </td><td>reflexively      </td></tr>
<tr><td>gram9-plural-verbs  </td><td>think      </td><td>thinks       </td><td>decrease    </td><td>decreases     </td><td>increase   </td><td>increases         </td><td>increase         </td></tr>
<tr><td>gram9-plural-verbs  </td><td>estimate   </td><td>estimates    </td><td>vanish      </td><td>vanishes      </td><td>disappear  </td><td>disappear         </td><td>disappear        </td></tr>
<tr><td>gram9-plural-verbs  </td><td>go         </td><td>goes         </td><td>talk        </td><td>talks         </td><td>talkpage   </td><td>mid-class         </td><td>reminisces       </td></tr>
<tr><td>gram3-comparative   </td><td>deep       </td><td>deeper       </td><td>bad         </td><td>worse         </td><td>good       </td><td>good              </td><td>uglier           </td></tr>
<tr><td>gram3-comparative   </td><td>simple     </td><td>simpler      </td><td>low         </td><td>lower         </td><td>higher     </td><td>higher            </td><td>hotter           </td></tr>
<tr><td>gram3-comparative   </td><td>sharp      </td><td>sharper      </td><td>bad         </td><td>worse         </td><td>nicer      </td><td>nicer             </td><td>prettier         </td></tr>
<tr><td>capital-world       </td><td>dublin     </td><td>ireland      </td><td>kingston    </td><td>jamaica       </td><td>england    </td><td>canada            </td><td>caledonia        </td></tr>
<tr><td>capital-world       </td><td>belmopan   </td><td>belize       </td><td>canberra    </td><td>australia     </td><td>tasmania   </td><td>melbourne         </td><td>fiji             </td></tr>
<tr><td>capital-world       </td><td>khartoum   </td><td>sudan        </td><td>lilongwe    </td><td>malawi        </td><td>tanzania   </td><td>tanzania          </td><td>congo-brazzaville</td></tr>
<tr><td>capital-common-count</td><td>hanoi      </td><td>vietnam      </td><td>london      </td><td>england       </td><td>1914-18    </td><td>islington         </td><td>africa           </td></tr>
<tr><td>capital-common-count</td><td>madrid     </td><td>spain        </td><td>baghdad     </td><td>iraq          </td><td>syria      </td><td>syria             </td><td>syria            </td></tr>
<tr><td>capital-common-count</td><td>canberra   </td><td>australia    </td><td>baghdad     </td><td>iraq          </td><td>syria      </td><td>mosul             </td><td>arabia           </td></tr>
</tbody>
</table>

In [9]:
analogies = defaultdict(list)
with open(analogy_filename, 'r') as f:
    for line in f:
        if line[0] == ":":
            topic = line.split()[-1]
        else:
            analogies[topic].append(tuple(line.split()))

In [51]:
def analogy_task(topics, analogies, embeddings):
    """Calculate the accuracy and MRR for embeddings on an analogy task.
    
    Args:
        analogies: dictionary with topics as keys and a list of word tuples as values
        embeddings: dictionary of word embeddings, words as keys and vectors as values
    Returns:
        float: accuracy
        float: MRR
    """
    accuracies = []
    mrrs = []
    all_mrr = 0
    all_correct = 0
    all_present = 0
    wrong_examples = defaultdict(list)
    for topic in tqdm(topics):
        topic_mrr = 0
        topic_correct = 0
        topic_present = 0
        for (a, a_star, b, b_star) in analogies[topic]:
            # If words with capital letters are not in the vectors, try to lowercase
            if a not in embeddings: a = a.lower()
            if a_star not in embeddings: a_star = a_star.lower()
            if b not in embeddings: b = b.lower()
            if b_star not in embeddings: b_star = b_star.lower()

            # If words from the analogy are not in the embeddings, do not let it count
            if any([word not in embeddings for word in [a, a_star, b, b_star]]):
                continue

            # Get the vector closest to the calculated analogy vector
            b_star_approx = embeddings[b] + (embeddings[a_star] - embeddings[a])
            b_star_approx = b_star_approx / np.linalg.norm(b_star_approx)
            words, similarities = zip(*list(embeddings.similar_by_vector(b_star_approx, topn=5000)))
            words = list(words)
            if b in words: words.remove(b)
            if a in words: words.remove(a)
            if a_star in words: words.remove(a_star)

            # Do not take b into account
            words = list(words)
            b_star_approx = words[0]

            # Calculate reciprocal rank for MRR
            reciprocal_rank = 1 / (words.index(b_star) + 1) if b_star in words else 0
            topic_mrr += reciprocal_rank
            all_mrr += reciprocal_rank

            # Calculate whether it's correct for Accuracy
            if b_star_approx == b_star:
                topic_correct += 1
                all_correct += 1
            else:
                wrong_examples[topic].append((a, a_star, b, b_star, words[:3]))
            topic_present += 1
            all_present += 1

        # Add Accuracy for this topic
        accuracies.append(round(topic_correct / topic_present, 2))

        # Add MRR for this topic
        mrrs.append(round(topic_mrr / topic_present, 2))

    # Calculate final Accuracy and MRR for all topics combined
    accuracies.append(round(all_correct / all_present, 2))
    mrrs.append(round(all_mrr / all_present, 2))
    return accuracies, mrrs, wrong_examples

topics = list(analogies.keys())
print("Analogy task for BOW2, this may take a while")
acc_bow2, mrr_bow2, wrong_bow2 = analogy_task(topics, analogies, bow2)

print("Analogy task for BOW5, this may take a while")
acc_bow5, mrr_bow5, wrong_bow5 = analogy_task(topics, analogies, bow5)

print("Analogy task for DEPS, this may take a while")
acc_deps, mrr_deps, wrong_deps = analogy_task(topics, analogies, deps)

Analogy task for BOW2, this may take a while


100%|██████████████████████████████████████████| 14/14 [09:58<00:00, 42.72s/it]


Analogy task for BOW5, this may take a while


100%|██████████████████████████████████████████| 14/14 [10:42<00:00, 45.91s/it]


Analogy task for DEPS, this may take a while


100%|██████████████████████████████████████████| 14/14 [09:32<00:00, 40.88s/it]


In [56]:
if create_figures:
    # List the accuracies and mrrs in a table for the quantitative analysis
    topics.append('All')
    results = [((len(analogies[line[0]]),) if line[0] != "All" 
               else (sum([len(analogies[t]) for t in analogies]),)) + line 
               for line in 
               zip(topics, acc_bow2, mrr_bow2, acc_bow5, mrr_bow5, acc_deps, mrr_deps) ]
    print(tabulate(results, headers=['Amount', 'Topic', 'Acc. Bow2', 'MRR Bow2',
                                     'Acc. Bow5', 'MRR Bow5',
                                     'Acc. Deps', 'MRR Deps'], tablefmt="html"))

    # Show samples for which all embeddings gave the wrong response
    wrong = defaultdict(lambda : defaultdict(dict))
    for topic in wrong_bow5:
        for analogy in wrong_bow5[topic]:
            wrong[topic][(analogy[0], analogy[1], analogy[2], analogy[3])]["bow5"] = analogy[4][0]

        for analogy in wrong_bow2[topic]:
            wrong[topic][(analogy[0], analogy[1], analogy[2], analogy[3])]["bow2"] = analogy[4][0]

        for analogy in wrong_deps[topic]:
            wrong[topic][(analogy[0], analogy[1], analogy[2], analogy[3])]["deps"] = analogy[4][0]

    wrong_table = []
    for topic in wrong:
        wrong_table.extend(list((topic[:20],) + item[0] + (item[1]["bow2"], item[1]["bow5"], item[1]["deps"]) 
                    for item in list(wrong[topic].items())
                    if all( embed in item[1] for embed in ["bow2", "bow5", "deps"]))[:3])
    print(tabulate(wrong_table, headers=['Topic', 'A', 'A*', 'B', 'B*', 'Bow2', 'Bow5', 'Deps'], tablefmt='html'))